# Extracting training data from the ODC

* **Products used:** 
[gm_s2_annual](https://explorer.digitalearth.africa/gm_s2_annual)


Source: https://registry.opendata.aws/deafrica-landsat/

We need to get the data for training the model as to get fit with, so that it can better predict the new test data that we feed later with high accuracy.
We will be using crop_training_egypt.geojson data for training the model. We use collect_training_data, a custom method from the Deafrica_tools package. This extracts the data that are geometry data type and is polygon specific. This function requires query parameters, zonal_stats(mean,max,min), feature layer function as parameters. Based on these params the data will be fetched and will be written into the physical file in the disk.

References
[Maxell et al 2018](https://www.tandfonline.com/doi/full/10.1080/01431161.2018.1433343)
[Geo-Wiki](https://www.geo-wiki.org/)
[Collect Earth Online](https://collect.earth/home)
[Radiant Earth](https://www.radiant.earth/)

## Getting started

We run the "Load packages" cell. 

### Load packages


In [2]:
%matplotlib inline

import os
import datacube
import numpy as np
import xarray as xr
import subprocess as sp
import geopandas as gpd
from odc.io.cgroups import get_cpu_quota
from datacube.utils.geometry import assign_crs

from deafrica_tools.plotting import map_shapefile
from deafrica_tools.bandindices import calculate_indices
from deafrica_tools.classification import collect_training_data

import warnings
warnings.filterwarnings("ignore")

## Analysis parameters

* `path`: The path to the input vector file from which we will extract training data. A default geojson is provided.
* `field`: This is the name of column in your shapefile attribute table that contains the class labels. **The class labels must be integers**


In [3]:
path = 'data/crop_training_egypt.geojson' 
field = 'class'

### Find the number of CPUs

In [4]:
ncpus=round(get_cpu_quota())
print('ncpus = '+str(ncpus))

ncpus = 2


## Preview input data

We can load and preview our input data shapefile using `geopandas`. 


In [5]:
# Load input data shapefile
input_data = gpd.read_file(path)

# Plot first five rows
input_data.head()

,class,geometry
0,0,"POLYGON ((26.19189 22.06193, 26.19230 22.06193..."
1,0,"POLYGON ((32.24947 22.07338, 32.24989 22.07338..."
2,0,"POLYGON ((32.62301 22.15862, 32.62342 22.15862..."
3,0,"POLYGON ((28.35345 22.29337, 28.35386 22.29337..."
4,0,"POLYGON ((27.72311 22.83994, 27.72352 22.83994..."


In [6]:
# Plot training data in an interactive map
map_shapefile(input_data, attribute=field)

Label(value='')

Map(center=[26.769110088616873, 29.937057490950004], controls=(ZoomControl(options=['position', 'zoom_in_text'…

## Extracting training data

In [7]:
#set up our inputs to collect_training_data
zonal_stats = 'mean'

# Set up the inputs for the ODC query
time = ('2019')
measurements =  ['blue','green','red','nir','swir_1','swir_2','red_edge_1',
                 'red_edge_2', 'red_edge_3', 'BCMAD', 'EMAD', 'SMAD']
resolution = (-20,20)
output_crs='epsg:6933'

Generate a datacube query object from the parameters above:

In [8]:
query = {
    'time': time,
    'measurements': measurements,
    'resolution': resolution,
    'output_crs': output_crs
}

## Defining feature layers

To create the desired feature layers, we pass instructions to `collect_training_data` through the `feature_func` parameter.

* `feature_func`: A function for generating feature layers that is applied to the data within the bounds of the input geometry. The `feature_func` must accept a `dc_query` dictionary, and return a single `xarray.Dataset` or `xarray.DataArray` containing 2D coordinates (i.e x, y - no time dimension). e.g.

          def feature_function(query):
              dc = datacube.Datacube(app='feature_layers')
              ds = dc.load(**query)
              ds = ds.mean('time')
              return ds

Below, we will define a more complicated feature layer function than the brief example shown above. We will calculate some band indices on the Sentinel-2 [geoMAD](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks/blob/master/Datasets/GeoMAD.ipynb) and append a slope dataset.



In [9]:
from datacube.testutils.io import rio_slurp_xarray

def feature_layers(query):
    #connect to the datacube
    dc = datacube.Datacube(app='feature_layers')
    
    #load s2 annual geomedian
    ds = dc.load(product='gm_s2_annual',
                 **query)
    
    #calculate some band indices
    da = calculate_indices(ds,
                           index=['NDVI', 'LAI', 'MNDWI'],
                           drop=False,
                           collection='s2')
    
    #add slope dataset
    url_slope = "https://deafrica-input-datasets.s3.af-south-1.amazonaws.com/srtm_dem/srtm_africa_slope.tif"
    slope = rio_slurp_xarray(url_slope, gbox=ds.geobox)
    slope = slope.to_dataset(name='slope')
    
    #merge results into single dataset 
    result = xr.merge([da, slope],compat='override')

    return result.squeeze()

Now let's run the `collect_training_data` function.

In [ ]:
column_names, model_input = collect_training_data(
                                    gdf=input_data,
                                    dc_query=query,
                                    ncpus=ncpus,
                                    field=field,
                                    zonal_stats=zonal_stats,
                                    feature_func=feature_layers
                                    )

Taking zonal statistic: mean


  0%|          | 0/156 [00:00<?, ?it/s]

## Export training data


In [12]:
#set the name and location of the output file
output_file = "results/test_training_data.txt"

In [13]:
#grab all columns
model_col_indices = [column_names.index(var_name) for var_name in column_names]
#Export files to disk
np.savetxt(output_file, model_input[:, model_col_indices], header=" ".join(column_names), fmt="%4f")

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Compatible datacube version:** 

In [14]:
print(datacube.__version__)

1.8.5


**Last Tested:**

In [1]:
from datetime import datetime
datetime.today().strftime('%Y-%m-%d')

'2022-05-04'